참조) https://mkjjo.github.io/python/2019/08/04/crypto.html

### AES256
- key: 32 bytes
- iv: 16 bytes
- block_size: 16 bytes

In [9]:
import base64
from Crypto import Random
from Crypto.Cipher import AES

In [266]:
import secrets

### VER1

In [354]:
class AES_cbcmode:
    
    def __init__(self,key):
        self.key = key
    
    def padding(self, raw_data):
        # 한국어의 경우 utf-8했을 때의 len이 달라지므로 encoding 과정 추가
        enc = raw_data.encode('utf-8')
        pad_num = AES.block_size - len(enc) % AES.block_size
        pad_chr = chr(pad_num)
        
        # encoding해서 최종 pad_data 반환
        return (raw_data + pad_num * pad_chr).encode('utf-8')
    
    def unpadding(self, pad_data):
        return pad_data[:-pad_data[-1]]
    
    def encrypt(self, raw_data):
        pad_data = self.padding(raw_data)
    
        # iv: initial vector
        # 두 번째 키와 같은 존재, 
        # 블록 단위로 암호/복호하면서 변경되고 그 값은 다음 블록 단위 암호/복호 시 사용
        iv = Random.new().read( AES.block_size )
        
        # CBC mode(Cipher Block Chaining)
        cipher = AES.new( self.key, AES.MODE_CBC, iv )
        return base64.b64encode(iv + cipher.encrypt(pad_data))
    
    def decrypt(self, enc_data):
        enc_decode = base64.b64decode(enc_data)
        iv = enc_decode[:AES.block_size]
        cipher = AES.new(self.key,AES.MODE_CBC, iv)
        return self.unpadding(cipher.decrypt(enc_decode[AES.block_size:]))

In [355]:
data="URL:https://python.flowdas.com/library/secrets.html#module-secrets\nDevice:dSmdf53621\nIP:184.51.65.127\nCode:dkjSDFsskedlfskdfjlks"

In [356]:
key = secrets.token_bytes(32)
key

b'\x94%\xf3"\x93\x9c,\x9f`\xcc\x94n\x1eh2Y{W\t\x9f7\xc2>l\x1cEx\xd4\x10\xfa\xecX'

In [357]:
enc_data = AES_cbcmode(key).encrypt(data)
enc_data

b'8llmuFhxkQMlh10yKoZy1K2F+QOPHB6jv8ddkn66qqB7LWg/WDPc/nNGSIoKvvfmw4JcSrniZczItqx7hJ2OrvY+BwfNnphf0nvQJw1CT4k0FginDMBXbgmDJnD+ykOgjmx5a+J+WO8mOvhXShsnC76WCiLQxFKFmnLxCxuhFBXXT7opNuxe/wu1JERRLg+sTW5VE9TUfu7fz3Gt6uLWpw=='

In [281]:
enc_data = AES_cbcmode(key).encrypt(data)
enc_data

b'b9kUoFV8aJZQu8wPZolrLX3GECLWssSNaJtVxad3i7KNOUmpOIL98WCg+BvLqlSpTdat3bKSotnB+qbXSaMDukcSo1pk30kytcjNl1hSgZN8Q43rX/kbpdpKbwlvH0Jc4+zhpuNtnUgDc0d5eGI89gyRXEH5VfyYR+ucDEiHN854TtjKYSdqpvNrUOLBHvhRy4UMukl6NqMKMytvr8fcqQ=='

In [358]:
dec_data = AES_cbcmode(key).decrypt(enc_data)
dec_data

b'URL:https://python.flowdas.com/library/secrets.html#module-secrets\nDevice:dSmdf53216\nIP:184.51.65.127\nCode:dkjSDFsskedlfskdfjlks'

### VER2

In [343]:
class AES_cbcmode_v2:
    
    def __init__(self,common_key,secret_key):
        self.key = common_key
        self.secret_key = secret_key
    
    def padding(self, raw_data):
        # 한국어의 경우 utf-8했을 때의 len이 달라지므로 encoding 과정 추가
        enc = raw_data.encode('utf-8')
        pad_num = AES.block_size - len(enc) % AES.block_size
        pad_chr = chr(pad_num)
        
        # encoding해서 최종 pad_data 반환
        return (raw_data + pad_num * pad_chr).encode('utf-8')
    
    def unpadding(self, pad_data):
        return pad_data[:-pad_data[-1]]
    
    def encrypt(self, raw_data, url):
        pad_data = self.padding(raw_data)
        pad_url = self.padding(url)
    
        # iv: initial vector
        # 두 번째 키와 같은 존재, 
        # 블록 단위로 암호/복호하면서 변경되고 그 값은 다음 블록 단위 암호/복호 시 사용
        iv = Random.new().read( AES.block_size)
        
        uiv = Random.new().read( AES.block_size)
        
        
        
        # CBC mode(Cipher Block Chaining)
        cipher_info = AES.new(self.secret_key, AES.MODE_CBC, iv)
        
        p_data = base64.b64encode(iv + cipher_info.encrypt(pad_data))
        
#         p_data = base64.b64encode(iv + cipher_info.encrypt(pad_data))
        
        
        
        cipher = AES.new(self.key, AES.MODE_CBC, uiv)
        
        return base64.b64encode(uiv + cipher.encrypt(pad_url + p_data))
    
    
    def url_decrypt(self, enc_data):
        enc_decode = base64.b64decode(enc_data)
        uiv = enc_decode[:AES.block_size]
        
        cipher = AES.new(self.key,AES.MODE_CBC, uiv)
        
        return self.unpadding(cipher.decrypt(enc_decode[AES.block_size:]))
    
    def decrypt(self, enc_data, u):
        enc_decode = base64.b64decode(enc_data)
        uiv = enc_decode[:AES.block_size]
        
        cipher = AES.new(self.key,AES.MODE_CBC, uiv)
        info = cipher.decrypt(enc_decode[AES.block_size:])
        
        iv = info[:AES.block_size]
        cipher_info = AES.new(self.secret_key,AES.MODE_CBC, iv)
        
        return self.unpadding(cipher_info.decrypt(cipher_info [AES.block_size:]))

In [306]:
url ="https://python.flowdas.com/library/secrets.html#module-secrets"
u ="python.flowdas.com"

In [307]:
secret_key = secrets.token_bytes(32)
secret_key

b'\x96p\xb3\x04\x04K1P\x94>\t\xe4\xdc\xf2\xd0\x146.\xb0/"\xd2\xfcuf\x9a\'\xa4\xe1\xd7\xb8\x00'

In [308]:
common_key = secrets.token_bytes(32)
common_key

b'\x06z\xd8GZi\xbd\x87\x1d\x8d\xd3Z\x176)\x1c(\xcbm8\xa0\xda\x19\xa8^\xe4u\x04\xf6R\xe6\xba'

In [316]:
data= "Device:dSmdf53216\nIP:184.51.65.127\nCode:dkjSDFsskedlfskdfjlks"

In [324]:
def padding(raw_data):
    # 한국어의 경우 utf-8했을 때의 len이 달라지므로 encoding 과정 추가
    enc = raw_data.encode('utf-8')
    pad_num = AES.block_size - len(enc) % AES.block_size
    pad_chr = chr(pad_num)

    # encoding해서 최종 pad_data 반환
    return (raw_data + pad_num * pad_chr).encode('utf-8')

pad_data = padding(data)
pad_url = padding(url)

In [325]:
print(pad_data)
print(pad_url)

b'Device:dSmdf53216\nIP:184.51.65.127\nCode:dkjSDFsskedlfskdfjlks\x03\x03\x03'
b'https://python.flowdas.com/library/secrets.html#module-secrets\x02\x02'


In [328]:

# iv: initial vector
# 두 번째 키와 같은 존재, 
# 블록 단위로 암호/복호하면서 변경되고 그 값은 다음 블록 단위 암호/복호 시 사용
iv = Random.new().read( AES.block_size)

uiv = Random.new().read( AES.block_size)

print(iv)
print(uiv)

b't\xa5@\xaf\x94\xc5\x16a\xeb\x82\x99g\xd9\x14e\xbc'
b'\xd5Qf\xa3+&\x06S?\xb1\xee\xda\xdf\x18\rl'


In [330]:

# CBC mode(Cipher Block Chaining)
cipher_info = AES.new(secret_key, AES.MODE_CBC, iv)


cipher = AES.new(common_key, AES.MODE_CBC, uiv)


In [336]:
p_data = iv + cipher_info.encrypt(pad_data)
pp_data = base64.b64encode(p_data)
pp_data

b'dKVAr5TFFmHrgpln2RRlvKPQ5LnPJx1T6Kp14iub1lPFY3cEtdcd4XtFqgoUv5PMOtcmygF0/sWySFs/PG2NIvTNOFPR2880bqDtwNkuFog='

In [337]:
ppp_data = str(pp_data,encoding="utf-8")
ppp_data

'dKVAr5TFFmHrgpln2RRlvKPQ5LnPJx1T6Kp14iub1lPFY3cEtdcd4XtFqgoUv5PMOtcmygF0/sWySFs/PG2NIvTNOFPR2880bqDtwNkuFog='

In [350]:
up = url+ppp_data
upp = padding(up)

In [353]:
cipher.encrypt(upp)

b'\x80\xfcpC\xb7&\xa1H\x0b\xc0C\x18W\xb4X\xdc\x89\x16:\xde\xc5\xd4a\xa4U\xd2\xd3\x19m\xe1\xc1\xec\x07\x7fX\xceLlp\xcbW<\x15\xfe\x16\xeeW\xef\xa3\x826v\xad\x98\x16\xaf\x8c\x1e\xf0\xf7\xfc\xe1\x0e\xa2#\xf6T\xec\xb8*\x88n6E\xab\x86\xdd\x10}\xfc\x9f+9#\xbd\x1c{\x8e3,\xfd\x8d\x90\xa1Z&\x0c\xa2m\xd4\xdbp\xc8\xab\xb1\x1c\x046\x0e\xa3\x0f\x1f\xe9q\xaeI\xb6\xc2\xb8]\xf7]2W\xbe\xa4\xd3\xab\xf2rF\x1a\x88\xbb\t\xfb\xfdT\xd8\xb4g\tH\x9cm\xe3\xf0\x05\x18F\x04\xa0\xb7\xa5\xbf\xe9\x9c\x81\x7f\x82{\xbd\xdb\x06\x0eR <KaC\xc3\xd2t\xa3\xb8'

In [341]:
url

'https://python.flowdas.com/library/secrets.html#module-secrets'

In [342]:
ppp_data

'dKVAr5TFFmHrgpln2RRlvKPQ5LnPJx1T6Kp14iub1lPFY3cEtdcd4XtFqgoUv5PMOtcmygF0/sWySFs/PG2NIvTNOFPR2880bqDtwNkuFog='

In [340]:
cipher.encrypt(url + ppp_data)

TypeError: Object type <class 'str'> cannot be passed to C code

In [ ]:
base64.b64encode(uiv + cipher.encrypt(url + p_data))

In [344]:
enc_data = AES_cbcmode_v2(common_key, secret_key).encrypt(data, url)
enc_data

ValueError: Data must be padded to 16 byte boundary in CBC mode

In [261]:
\xdcrc

SyntaxError: unexpected character after line continuation character (<ipython-input-261-548a43e5b388>, line 1)

In [255]:
ka = Random.new().read( 32 )
ka

b'\xdcrc\xe5\x01\xb5\xb7\xfa\xc4r\xf9\xc1tg?\x83v\xd6{\xe30\xa79[P\xb9\nV2ZP='

In [262]:
ka[0:3]

b'\xdcrc'

In [264]:
ka[0:3].decode()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xdc in position 0: invalid continuation byte

In [259]:
ii = 0
for i in ka:
    print(i)
    print(ka[ii])
    ii += 1

220
220
114
114
99
99
229
229
1
1
181
181
183
183
250
250
196
196
114
114
249
249
193
193
116
116
103
103
63
63
131
131
118
118
214
214
123
123
227
227
48
48
167
167
57
57
91
91
80
80
185
185
10
10
86
86
50
50
90
90
80
80
61
61


In [251]:
edd = AES_cbcmode(bytes(ka)).encrypt(data)
edd

b'e2ptX9zHdoAuSRKgfZyRLmJt6k8LSK9CG6Ojw2Zzpr3CPHHMG2S8G1yCqF49talexYdtQMbu4sE2VsBwh6kX8UmAZTaG+vTVbzTRDIBbpaQ='

In [254]:
str(AES_cbcmode(bytes(ka)).decrypt(edd), encoding = "utf-8")

'i have a butterfly in my stomache. 인생은 요지경'

In [ ]:
edd = AES_cbcmode(bytes(ka)).encrypt(data)
edd

In [241]:
k = [16, 1, 21, 27, 161, 255, 300]

In [226]:
kk = [0x10, 0x01, 0x15, 0x1B, 0xA1, 0x11]

In [235]:
kkd = [0x10, 0x1, 0x15, 0x1B, 0xA1, 0x11]

In [236]:
kkd

[16, 1, 21, 27, 161, 17]

In [240]:
hex(256)

'0x100'

In [227]:
bytes(kk)

b'\x10\x01\x15\x1b\xa1\x11'

In [242]:
bytes(k)

ValueError: bytes must be in range(0, 256)

In [248]:
bytes([255])

b'\xff'

In [ ]:
bytes(k)

In [206]:
print(key)

[16, 1, 21, 27, 161, 17, 87, 114, 108, 33, 86, 87, 98, 22, 5, 61, 255, 254, 17, 27, 33, 49, 87, 114, 107, 33, 166, 167, 110, 230, 229, 63]


In [203]:
key = [0x10, 0x01, 0x15, 0x1B, 0xA1, 0x11, 0x57, 0x72, 0x6C, 0x21, 0x56, 0x57, 0x62, 0x16, 0x05, 0x3D,
        0xFF, 0xFE, 0x11, 0x1B, 0x21, 0x31, 0x57, 0x72, 0x6B, 0x21, 0xA6, 0xA7, 0x6E, 0xE6, 0xE5, 0x3F]

In [195]:
data = "i have a butterfly in my stomache. 인생은 요지경"

In [196]:
encd = AES_cbcmode(bytes(key)).encrypt(data)
encd

b'nHUPabAHkOfoJrPG8ZA55amRg7bS+1xH3br9OFc5JCrz5nJQIIOeCOt02hpqpoOlhLNd5myvZovnOC6YYJM4bkn0JuO9enynwvBh1vtqAj8='

In [198]:
ddd = AES_cbcmode(bytes(key)).decrypt(encd)

In [199]:
str(ddd, encoding = 'utf-8')

'i have a butterfly in my stomache. 인생은 요지경'

In [170]:
c = [1,2,3,4,5]

In [131]:
iv = Random.new().read( AES.block_size )

In [132]:
type(iv)

bytes

In [134]:
pad_data.encode('utf-8')+iv

b'ddd\r\r\r\r\r\r\r\r\r\r\r\r\rN\x0c \xa0lLV\n\xe9\xf5\x80M\x84\xf0\xdf\xf0'

In [148]:
d = base64.b64encode(iv + pad_data.encode('utf-8'))

In [149]:
ord(chr(10))

10

In [150]:
e = base64.b64decode(d)

In [151]:
e[16:]

b'ddd\r\r\r\r\r\r\r\r\r\r\r\r\r'

In [157]:
ee = e[16:]
ee

b'ddd\r\r\r\r\r\r\r\r\r\r\r\r\r'

In [167]:
str(eee,encoding='utf-8')

'ddd'

In [169]:
eee = ee[: -ee[-1]]
eee

b'ddd'

In [163]:
ord(ee[len(ee)-1:])

13

In [4]:
BS = 16
pad = lambda s: s + (BS - len(s.encode('utf-8')) % BS) * chr(BS - len(s.encode('utf-8')) % BS)
unpad = lambda s : s[:-ord(s[len(s)-1:])]

In [15]:
AES.block_size

16

In [32]:
class AES:
    def __init__( self, key ):
        self.key = key

    def encrypt( self, raw ):
        raw = pad(raw)
        
        # iv: initial vector
        # 두 번째 키와 같은 존재, 
        # 블록 단위로 암호/복호하면서 변경되고 그 값은 다음 블록 단위 암호/복호 시 사용
        
        iv = Random.new().read( AES.block_size )
        
        # CBC mode(Cipher Block Chaining)
        cipher = AES.new( self.key, AES.MODE_CBC, iv )
        return base64.b64encode( iv + cipher.encrypt( raw.encode('utf-8') ) )

    def decrypt( self, enc ):
        enc = base64.b64decode(enc)
        iv = enc[:16]
        cipher = AES.new(self.key, AES.MODE_CBC, iv )
        return unpad(cipher.decrypt( enc[16:] ))

In [27]:
key = [0x10, 0x01, 0x15, 0x1B, 0xA1, 0x11, 0x57, 0x72, 0x6C, 0x21, 0x56, 0x57, 0x62, 0x16, 0x05, 0x3D,
        0xFF, 0xFE, 0x11, 0x1B, 0x21, 0x31, 0x57, 0x72, 0x6B, 0x21, 0xA6, 0xA7, 0x6E, 0xE6, 0xE5, 0x3F]

In [28]:
key2 = [0x01, 0x01, 0x15, 0x1B, 0xA1, 0x11, 0x57, 0x72, 0x6C, 0x21, 0x56, 0x57, 0x62, 0x16, 0x05, 0x3D,
        0xFF, 0xFE, 0x11, 0x1B, 0x21, 0x31, 0x57, 0x72, 0x6B, 0x21, 0xA6, 0xA7, 0x6E, 0xE6, 0xE5, 0x3F]

In [29]:
data = "dddddddddhhhhhhhhhhhhh"

In [33]:
encrypted_data = AESCipher(bytes(key)).encrypt(data)  
encrypted_data

TypeError: bytes() argument 2 must be str, not list

In [34]:
decrypted_data = AESCipher(bytes(key)).decrypt(encrypted_data)
decrypted_data.decode('utf-8')

TypeError: bytes() argument 2 must be str, not list

# ===================================

In [ ]:
import base64
from Crypto.Cipher import AES
from Crypto import Random

class AESCipher:
    def __init__( self, key ):
        self.key = key

    def encrypt( self, raw ):
        raw = pad(raw)
        iv = Random.new().read( AES.block_size )
        cipher = AES.new( self.key, AES.MODE_CBC, iv )
        return base64.b64encode( iv + cipher.encrypt( raw ) ) 

    def decrypt( self, enc ):
        enc = base64.b64decode(enc)
        iv = enc[:16]
        cipher = AES.new(self.key, AES.MODE_CBC, iv )
        return unpad(cipher.decrypt( enc[16:] ))

In [59]:
from Crypto.Cipher import AES
from Crypto.Util import Counter
from Crypto import Random

# AES supports multiple key sizes: 16 (AES128), 24 (AES192), or 32 (AES256).
key_bytes = 32

# Takes as input a 32-byte key and an arbitrary-length plaintext and returns a
# pair (iv, ciphtertext). "iv" stands for initialization vector.
def encrypt(key, plaintext):
    assert len(key) == key_bytes

    # Choose a random, 16-byte IV.
    iv = Random.new().read(AES.block_size)

    # Convert the IV to a Python integer.
    iv_int = int(binascii.hexlify(iv), 16) 

    # Create a new Counter object with IV = iv_int.
    ctr = Counter.new(AES.block_size * 8, initial_value=iv_int)

    # Create AES-CTR cipher.
    aes = AES.new(key, AES.MODE_CTR, counter=ctr)

    # Encrypt and return IV and ciphertext.
    ciphertext = aes.encrypt(plaintext)
    return (iv, ciphertext)

# Takes as input a 32-byte key, a 16-byte IV, and a ciphertext, and outputs the
# corresponding plaintext.
def decrypt(key, iv, ciphertext):
    assert len(key) == key_bytes

    # Initialize counter for decryption. iv should be the same as the output of
    # encrypt().
    iv_int = int(iv.encode('hex'), 16) 
    ctr = Counter.new(AES.block_size * 8, initial_value=iv_int)

    # Create AES-CTR cipher.
    aes = AES.new(key, AES.MODE_CTR, counter=ctr)

    # Decrypt and return the plaintext.
    plaintext = aes.decrypt(ciphertext)
    return plaintext

(iv, ciphertext) = encrypt(key, 'hella')
print decrypt(key, iv, ciphertext)

SyntaxError: invalid syntax (<ipython-input-59-98cc88445a4f>, line 47)

# 예시 연습

In [11]:
import base64
import time
import hashlib

def make_pass():
    timekey = int(time.time())
    print(timekey)
    return str(timekey)

password = make_pass().encode('utf-8')
print(password)

1586539553
b'1586539553'


In [3]:
password

b'1586539411'

In [4]:
make_pass()

'1586539421'

In [5]:
60*60*24*30*12

31104000

In [7]:
60*60*24*30*12*100

3110400000

In [8]:
key = [0x10, 0x01, 0x15, 0x1B, 0xA1, 0x11, 0x57, 0x72, 0x6C, 0x21, 0x56, 0x57, 0x62, 0x16, 0x05, 0x3D,
        0xFF, 0xFE, 0x11, 0x1B, 0x21, 0x31, 0x57, 0x72, 0x6B, 0x21, 0xA6, 0xA7, 0x6E, 0xE6, 0xE5, 0x3F]

In [9]:
key

[16,
 1,
 21,
 27,
 161,
 17,
 87,
 114,
 108,
 33,
 86,
 87,
 98,
 22,
 5,
 61,
 255,
 254,
 17,
 27,
 33,
 49,
 87,
 114,
 107,
 33,
 166,
 167,
 110,
 230,
 229,
 63]